# Activities

This example shows how to create an activity

In [ ]:
import ovation as ov
import ovation.activities as activities

from ovation.session import connect
from tqdm import tqdm_notebook
from pprint import pprint

In [ ]:
ov.__version__

In [ ]:
s = connect(input('Email: '), org=int(input("Organization (enter for default): ") or 0))

## Create a new activity

When creating an activity, you can specify the inputs, outputs and related files at the time of creation. But don't worry, you can also add and remove inputs, outputs, and related files later.

Activity inputs are specified as an array of UUIDs or entity `dicts` and can be either `Revisions` or `Sources`. You can use the local path of a file. Local files will be uploaded (creating associated `File` and `Reivision` records).

In [ ]:
inputs = ['181c9eb7-8450-4d59-9b5a-ede7fb984b51','181c9eb7-8450-4d59-9b5a-ede7fb984b51'] 

Activity outputs are specified as an array of UUIDs or entity `dicts` to `Revisions`. You can use the local path of a file as well. Local files will be uploaded (creating associated `File` and `Reivision` records).

In [ ]:
outputs = ['9744f67f-7daa-43a5-901f-8f63b5b956d4']

`create_activity` creates the activity and associates it with the given `Project`. 

In [ ]:
project_id = input('Project UUID: ')
activity_name = input('Activity name: ')
activity = activities.create_activity(s,
                                      project_id,
                                      activity_name, 
                                      inputs=inputs, outputs=outputs)

## Add outputs

After creation, you can modify the inputs, outputs, and related files of an Activity. Of course, it's a good idea to do this carefully if downstream results depend on the results of this Activity. Inputs, outputs, and related files are added and removed in the same way (using `add_inputs`, `remove_inputs`; `add_outputs`, `remove_outputs`; `add_related`, and `remove_related`). This example shows adding a file as a input:

In [ ]:
activites.add_inputs(s, activity, inputs=['local/file/path.txt'])

To remove an `input`, use `remove_inputs`:

In [ ]:
activites.remove_inputs(s, activity, inputs=['181c9eb7-8450-4d59-9b5a-ede7fb984b51'])